<a href="https://colab.research.google.com/github/KimDonghwi96/ChineseTaiwanese-FilmReview/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94_%EB%8C%93%EA%B8%80_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 6.8MB/s 
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1E2RtqkmRMI53j8IQGslP3XNmO0S36ccN

rawdata_downloaded = drive.CreateFile({'id': '1E2RtqkmRMI53j8IQGslP3XNmO0S36ccN'})
rawdata_downloaded.GetContentFile('진짜2.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('진짜2.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,number,type,text,point,nickname,score,like,dislike,time
0,0,5일의마중,길고 긴 여운을 깰 자신이 없는 영화,연기,임치즈(mimi****),10.0,208.0,18.0,2014-10-08 17:44
1,1,5일의마중,"상대의 눈에 비치는 내 모습이 아닌, 그 사람 자체를 사랑하는 것이 어떤 것인지를 ...",연기,lumiere428(ally****),10.0,189.0,17.0,2014-10-08 13:16
2,2,5일의마중,"영화는 절대 먼저 울지 않는데, 보는 사람은 가슴이 미어진다.",연기,jimmani(jimm****),10.0,147.0,10.0,2014-10-09 0:12
3,3,5일의마중,올해본 영화중에 최고. 영화관에서 얼마나 눈물을 쏟아냈는지 모른다.,연기,그레이스(ksw3****),10.0,121.0,13.0,2014-10-08 21:48
4,4,5일의마중,공리...우와...,연기,voirmonparadis(core****),10.0,113.0,6.0,2014-10-09 13:30
...,...,...,...,...,...,...,...,...,...
54509,54510,패왕별희,장국영의 게이 연기 너무나도 역겨웠다,연기/연출/스토리,우주최강(rlat****),10.0,2.0,2.0,2020-05-02 22:30
54510,54511,패왕별희,경극 소리 시끄럽다.그리고 중꿔들 왜 그렇게 벚고 다님?,연기/연출/스토리,코무(gkdl****),10.0,2.0,2.0,2020-05-02 18:47
54511,54512,패왕별희,아무리 명작이라도 짱깨는 거르자,연기/연출/스토리,김채원(chaw****),10.0,2.0,2.0,2020-05-02 17:47
54512,54513,패왕별희,와 이건 1점도 아깝다 진짜 재미없고 지루한 영화ㅡㅡ 여기 진짜 거의다 중국인들이 ...,연기/연출/스토리,oay2****,10.0,2.0,2.0,2020-05-02 11:13


In [ ]:
원본데이터 = 원본데이터.dropna(axis=0)

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,number,type,text,point,nickname,score,like,dislike,time
0,0,5일의마중,길고 긴 여운을 깰 자신이 없는 영화,연기,임치즈(mimi****),10.0,208.0,18.0,2014-10-08 17:44
1,1,5일의마중,"상대의 눈에 비치는 내 모습이 아닌, 그 사람 자체를 사랑하는 것이 어떤 것인지를 ...",연기,lumiere428(ally****),10.0,189.0,17.0,2014-10-08 13:16
2,2,5일의마중,"영화는 절대 먼저 울지 않는데, 보는 사람은 가슴이 미어진다.",연기,jimmani(jimm****),10.0,147.0,10.0,2014-10-09 0:12
3,3,5일의마중,올해본 영화중에 최고. 영화관에서 얼마나 눈물을 쏟아냈는지 모른다.,연기,그레이스(ksw3****),10.0,121.0,13.0,2014-10-08 21:48
4,4,5일의마중,공리...우와...,연기,voirmonparadis(core****),10.0,113.0,6.0,2014-10-09 13:30
...,...,...,...,...,...,...,...,...,...
54509,54510,패왕별희,장국영의 게이 연기 너무나도 역겨웠다,연기/연출/스토리,우주최강(rlat****),10.0,2.0,2.0,2020-05-02 22:30
54510,54511,패왕별희,경극 소리 시끄럽다.그리고 중꿔들 왜 그렇게 벚고 다님?,연기/연출/스토리,코무(gkdl****),10.0,2.0,2.0,2020-05-02 18:47
54511,54512,패왕별희,아무리 명작이라도 짱깨는 거르자,연기/연출/스토리,김채원(chaw****),10.0,2.0,2.0,2020-05-02 17:47
54512,54513,패왕별희,와 이건 1점도 아깝다 진짜 재미없고 지루한 영화ㅡㅡ 여기 진짜 거의다 중국인들이 ...,연기/연출/스토리,oay2****,10.0,2.0,2.0,2020-05-02 11:13


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,길고,Adjective,5일의마중,1
1,긴,Verb,5일의마중,1
2,여운,Noun,5일의마중,1
3,을,Josa,5일의마중,1
4,깰,Verb,5일의마중,1
...,...,...,...,...
11,보다가,Verb,패왕별희,1
12,토,Noun,패왕별희,1
13,할,Verb,패왕별희,1
14,뻔해서,Adjective,패왕별희,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,길고,Adjective,5일의마중,1
1,긴,Verb,5일의마중,1
2,여운,Noun,5일의마중,1
3,을,Josa,5일의마중,1
4,깰,Verb,5일의마중,1
...,...,...,...,...
11,보다가,Verb,패왕별희,1
12,토,Noun,패왕별희,1
13,할,Verb,패왕별희,1
14,뻔해서,Adjective,패왕별희,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소      품사       분류             
        Foreign  나의 소녀시대            1
       Foreign  나의 소녀시대            1
        Foreign  그시절 우리가 좋아했던 소녀    2
                  시절인연               1
        Foreign  말할 수 없는 비밀         1
                                    ..
不能說的秘密   Foreign  말할 수 없는 비밀         3
＊        Foreign  말할 수 없는 비밀         3
，        Foreign  그시절 우리가 좋아했던 소녀    1
，又不是女嬌娥  Foreign  패왕별희               1
＠        Foreign  그시절 우리가 좋아했던 소녀    1
Name: 카운트, Length: 61634, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소     품사      분류                  
       Foreign 나의 소녀시대            1
      Foreign 나의 소녀시대            1
       Foreign 그시절 우리가 좋아했던 소녀    2
                시절인연               1
       Foreign 말할 수 없는 비밀         1
...                              ...
不能說的秘密  Foreign 말할 수 없는 비밀         3
＊       Foreign 말할 수 없는 비밀         3
，       Foreign 그시절 우리가 좋아했던 소녀    1
，又不是女嬌娥 Foreign 패왕별희               1
＠       Foreign 그시절 우리가 좋아했던 소녀    1

[61634 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')